### Many-to-Many Relations:

Lets use what have already. It is logical to say that that a movie could be translated to many laguages, and also there could be many movies in a single language.<br>
lets create this relation...<br>

1- Create a new data-model called Language which has only one attribute called name which has the CharField type with max length of 50.<br>
2- Redefine its str method.<br>
3- Create the neccesary class and methods in the admin.py file so that we can later manage it inside the adminstrative section.<br>

### Now, lets add the relation...

- Inside your Movie class, define a new attribute called translated_to and set it to models.ManyToManyField() and pass it the name of the model we created earlier.<br>
An important thing to mention here is that, with Many-to-Many relations, you can not set an on_delete behavior, the reason I think is obvious.

Your models.py file should look like the following block:

In [ ]:
from django.db import models
from django.core.validators import MinValueValidator, MaxValueValidator
from django.urls import reverse
from django.utils.text import slugify


class Language(models.Model):
    name = models.CharField(max_length=50)

    def __str__(self):
        return self.name


class Address(models.Model):
    street = models.CharField(max_length=80)
    postal_code = models.CharField(max_length=12)
    city = models.CharField(max_length=50)

    def full_address(self):
        return f"Street: {self.street}, Postal Code: {self.postal_code}, City: {self.city}"

    def __str__(self):
        return self.full_address()
    
    class Meta:
        verbose_name_plural = 'Address Entries'


class Actor(models.Model):
    first_name = models.CharField(max_length=100)
    last_name  = models.CharField(max_length=100)
    address = models.OneToOneField(Address, on_delete=models.CASCADE, null=True)

    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    def __str__(self):
        return self.full_name()

class Movie(models.Model):
    title = models.CharField(max_length=50)
    rating = models.IntegerField(validators=[MinValueValidator(1), MaxValueValidator(10)])
    main_act= models.ForeignKey(Actor, on_delete=models.CASCADE, null=True, related_name='movies')
    is_bestselling = models.BooleanField(default=False)
    slug = models.SlugField(default='', null=False, db_index=True)
    translated_to = models.ManyToManyField(Language)

    def get_absolute_url(self):
        return reverse("movie_detail_url", args=[self.slug])

    def __str__(self):
        return f"title:{self.title}, rating:{self.rating}, Main Actor/actress:{self.main_act}{', Best Seller' if self.is_bestselling else ''}"

Your admin.py file should look like the following block:

In [ ]:
from django.contrib import admin
from .models import Movie, Actor, Address, Language


class LanguageAdmin(admin.ModelAdmin):
    list_display = ['name']

class AddressAdmin(admin.ModelAdmin):
    list_display = ['street', 'postal_code', 'city']

class ActorAdmin(admin.ModelAdmin):
    list_display = ['first_name', 'last_name', 'address']

class MovieAdmin(admin.ModelAdmin):
    prepopulated_fields = {'slug':('title',)}
    list_filter = ('rating', 'is_bestselling')
    list_display = ['title', 'main_act']

admin.site.register(Movie, MovieAdmin)
admin.site.register(Actor, ActorAdmin)
admin.site.register(Address, AddressAdmin)
admin.site.register(Language, LanguageAdmin)

Lets add some data to the language data-model and then use them in the Movie data-model<br>

Before you do anything you need to makemigrations and then migrate.

Here are the codes you many use:

In [ ]:
farsi = Language(name='Farsi')
farsi.save()

english = Language(name="English")
english.save()

french = Language(name='French')
french.save()

hp1 = Movie.objects.get(title='Harry Potter and The Philosopher\'s Stone')
the_dictator = Movie.objects.get(title='The Dictator')

movie1.translated_to.add(english,french,farsi)
Movie2.translated_to.add(farsih)

As you may have noticed, we can not just assing a many-to-many field to a value like the other relations that we had. Insted you need to use the .add() method and then send the values.<br>
You could also use .set and send all the values inside a list if you want to change the whole values for this field.

Something important to note is that all the avalible values for the translate_to field will be displayed on the admin page! however only those which are highlighted are the acutal values.<br>
If you wish to change the highlighted values press ctrl+click on the values that you want (don't forget to save)

Add a meta class to the Language model so that instead of languages the name 'All Languages' gets displayed on the admin page.

In [ ]:
class Language(models.Model):
    name = models.CharField(max_length=50)

    def __str__(self):
        return self.name
    
    class Meta:
        verbose_name_plural = 'All Languages'

### End of Chapter 5